## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from G_key import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Abasha,GE,42.2096,42.2211,53.82,66,5,10.38,clear sky
1,Tiksi,RU,71.6872,128.8694,7.63,90,26,6.55,scattered clouds
2,Hermanus,ZA,-34.4187,19.2345,55.80,85,5,12.19,clear sky
3,Vaini,TO,-21.2000,-175.2000,84.36,79,75,5.75,broken clouds
4,Provideniya,RU,64.3833,-173.3000,32.59,96,100,12.01,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,Vaini,TO,-21.2000,-175.2000,84.36,79,75,5.75,broken clouds
6,Puerto Ayora,EC,-0.7393,-90.3518,75.38,92,71,4.99,broken clouds
8,Mandera,KE,3.9366,41.8670,81.25,68,73,10.65,broken clouds
9,New Norfolk,AU,-42.7826,147.0587,78.89,37,100,3.00,overcast clouds
10,Hilo,US,19.7297,-155.0900,72.28,92,100,5.75,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   381
Country                378
Lat                    381
Lng                    381
Max Temp               381
Humidity               381
Cloudiness             381
Wind Speed             381
Current Description    381
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City                   378
Country                378
Lat                    378
Lng                    378
Max Temp               378
Humidity               378
Cloudiness             378
Wind Speed             378
Current Description    378
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

/var/folders/q1/_ty3whwx34q98jzpxnjtd2pr0000gn/T/ipykernel_16959/868910814.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
6,Puerto Ayora,EC,75.38,broken clouds,-0.7393,-90.3518,
8,Mandera,KE,81.25,broken clouds,3.9366,41.8670,
9,New Norfolk,AU,78.89,overcast clouds,-42.7826,147.0587,
10,Hilo,US,72.28,moderate rain,19.7297,-155.0900,
11,Butaritari,KI,82.06,scattered clouds,3.0707,172.7902,
14,Taltal,CL,61.11,clear sky,-25.4000,-70.4833,
15,Birjand,IR,63.07,dust,32.8663,59.2211,
16,Gorontalo,ID,76.35,overcast clouds,0.5412,123.0595,
19,Yangi Marg`ilon,UZ,66.92,overcast clouds,40.4272,71.7189,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Puerto Ayora,EC,75.38,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Mandera,KE,81.25,broken clouds,3.9366,41.8670,Hiren Hotel Mandera
9,New Norfolk,AU,78.89,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
10,Hilo,US,72.28,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
686,Mbaïki,CF,71.13,broken clouds,3.8678,17.9892,Motel MBM
691,San Javier,ES,62.33,clear sky,37.8063,-0.8374,Hotel Mar Menor
692,Dori,BF,83.16,overcast clouds,14.0354,-0.0345,Hôtel Liptako
693,Kandi,BJ,79.52,broken clouds,11.1342,2.9386,Baric Motel De Kandi


In [12]:
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)

In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
6,Puerto Ayora,EC,75.38,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
8,Mandera,KE,81.25,broken clouds,3.9366,41.8670,Hiren Hotel Mandera
9,New Norfolk,AU,78.89,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
10,Hilo,US,72.28,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
684,Bragança,BR,73.44,overcast clouds,-1.0536,-46.7656,Marujo's Suíte Hotel
686,Mbaïki,CF,71.13,broken clouds,3.8678,17.9892,Motel MBM
691,San Javier,ES,62.33,clear sky,37.8063,-0.8374,Hotel Mar Menor
692,Dori,BF,83.16,overcast clouds,14.0354,-0.0345,Hôtel Liptako


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations.head()

,Lat,Lng
3,-21.2000,-175.2000
6,-0.7393,-90.3518
8,3.9366,41.8670
9,-42.7826,147.0587
10,19.7297,-155.0900


In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))